In [6]:
# pip install pandarallel pymystem3

In [1]:
from tqdm.notebook import tqdm_notebook
from tqdm import tqdm

import pandas as pd
tqdm_notebook.pandas()
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)
import string

from pymystem3 import Mystem
import time

from collections import Counter
import itertools 

import nltk
# nltk.download('stopwords')
from nltk.corpus import stopwords

INFO: Pandarallel will run on 2 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


## Очистка данных и обработка текстов

Удаляем наны в переменной text и новости, датируемые 1914 годом

Поскольку наши данные представлены в виде связного текста, нам необходимо преобразовать их в понятный для машины язык - набор векторов. Перед этим необходимо провести обработку всех текстов и преобразовать их в массивы слов в начальной форме.

Мы протестировали несколько библиотек, которые используются для обработки текстов, одна из которых - Mystem. Эта библиотека разработана Яндексом, и ее особенность заключается в том, что она учитывает контекст слова. Также она самостоятельно разбивает текст на токены

Также нам необходимо убрать "стоп-слова"

In [114]:
stopwords_rus = stopwords.words('russian')
stopwords_rus

['и',
 'в',
 'во',
 'не',
 'что',
 'он',
 'на',
 'я',
 'с',
 'со',
 'как',
 'а',
 'то',
 'все',
 'она',
 'так',
 'его',
 'но',
 'да',
 'ты',
 'к',
 'у',
 'же',
 'вы',
 'за',
 'бы',
 'по',
 'только',
 'ее',
 'мне',
 'было',
 'вот',
 'от',
 'меня',
 'еще',
 'нет',
 'о',
 'из',
 'ему',
 'теперь',
 'когда',
 'даже',
 'ну',
 'вдруг',
 'ли',
 'если',
 'уже',
 'или',
 'ни',
 'быть',
 'был',
 'него',
 'до',
 'вас',
 'нибудь',
 'опять',
 'уж',
 'вам',
 'ведь',
 'там',
 'потом',
 'себя',
 'ничего',
 'ей',
 'может',
 'они',
 'тут',
 'где',
 'есть',
 'надо',
 'ней',
 'для',
 'мы',
 'тебя',
 'их',
 'чем',
 'была',
 'сам',
 'чтоб',
 'без',
 'будто',
 'чего',
 'раз',
 'тоже',
 'себе',
 'под',
 'будет',
 'ж',
 'тогда',
 'кто',
 'этот',
 'того',
 'потому',
 'этого',
 'какой',
 'совсем',
 'ним',
 'здесь',
 'этом',
 'один',
 'почти',
 'мой',
 'тем',
 'чтобы',
 'нее',
 'сейчас',
 'были',
 'куда',
 'зачем',
 'всех',
 'никогда',
 'можно',
 'при',
 'наконец',
 'два',
 'об',
 'другой',
 'хоть',
 'после',
 'на

In [115]:
m = Mystem()

In [9]:
def make_lemmas(x):
    """
    Лемматизируем строку, проверяя следующие моменты:
        - не стоп-слово
        - длина больше 2х символов
        - состоит только из букв либо слова, которые пишутся через тире
    """
    x = str(x)
    lemmas = m.lemmatize(x)
    text = [i.lower() for i in lemmas if i not in stopwords_rus and len(i) > 2
            and (i.isalpha() or ((i.split('-')[0]).isalpha()))]
    return text

Запустим один раз, чтобы проверить время работы 2х раз

In [12]:
%%time

df_iterator = pd.read_csv("/Users/kucumovamilana/Downloads/lenta-ru-news-with-2022.csv", chunksize=10000)
chunk = 1
for df in df_iterator:
    df['date'] = pd.to_datetime(df['date'])
    df = df[~df['text'].isna()]
    df = df[df['date'].dt.year != 1914]
    print(df)
    print(df.shape)
    df['text'] = df['text'].parallel_apply(make_lemmas)
    if chunk == 1:
        df.to_csv('lenta_news_with_lemmas.csv')
    else:
        df.to_csv('lenta_news_with_lemmas.csv', mode='a', header=False)
        break
    chunk += 1

           date                                            url  \
5    1999-08-31  https://lenta.ru/news/1999/08/31/stancia_mir/   
6    1999-08-31        https://lenta.ru/news/1999/08/31/vzriv/   
7    1999-08-31  https://lenta.ru/news/1999/08/31/credit_japs/   
8    1999-08-31        https://lenta.ru/news/1999/08/31/diana/   
9    1999-08-31          https://lenta.ru/news/1999/08/31/mvf/   
...         ...                                            ...   
9995 2000-07-10        https://lenta.ru/news/2000/07/10/zenit/   
9996 2000-07-10        https://lenta.ru/news/2000/07/10/metro/   
9997 2000-07-10     https://lenta.ru/news/2000/07/10/naemniki/   
9998 2000-07-10      https://lenta.ru/news/2000/07/10/tallinn/   
9999 2000-07-10   https://lenta.ru/news/2000/07/10/alexandrov/   

               topic tags                                              title  \
5             Россия  Все         Космонавты сомневаются в надежности "Мира"   
6             Россия  Все       Взрыв в центре 

            date                                            url      topic  \
10000 2000-07-10    https://lenta.ru/news/2000/07/10/explosive/     Россия   
10001 2000-07-10  https://lenta.ru/news/2000/07/10/vladikavkaz/     Россия   
10002 2000-07-10        https://lenta.ru/news/2000/07/10/tacis/  Экономика   
10003 2000-07-10          https://lenta.ru/news/2000/07/10/imf/  Экономика   
10004 2000-07-10       https://lenta.ru/news/2000/07/10/manila/        Мир   
...          ...                                            ...        ...   
19995 2001-01-17         https://lenta.ru/news/2001/01/17/case/     Россия   
19996 2001-01-17      https://lenta.ru/news/2001/01/17/clinton/        Мир   
19997 2001-01-17        https://lenta.ru/news/2001/01/17/truck/        Мир   
19998 2001-01-17         https://lenta.ru/news/2001/01/17/duma/     Россия   
19999 2001-01-17          https://lenta.ru/news/2001/01/17/vip/  Экономика   

      tags                                              title  

CPU times: user 6 s, sys: 1.09 s, total: 7.09 s
Wall time: 2min 14s


Проверим получившуюся csv

In [13]:
df_new = pd.read_csv('lenta_news_with_lemmas.csv')
df_new

,Unnamed: 0,date,url,topic,tags,title,text
0,5,1999-08-31,https://lenta.ru/news/1999/08/31/stancia_mir/,Россия,Все,"Космонавты сомневаются в надежности ""Мира""","['становиться', 'известно', 'агентство', 'ассо..."
1,6,1999-08-31,https://lenta.ru/news/1999/08/31/vzriv/,Россия,Все,Взрыв в центре Москвы: пострадало 30 человек,"['зал', 'игровой', 'автомат', 'третий', 'ярус'..."
2,7,1999-08-31,https://lenta.ru/news/1999/08/31/credit_japs/,Россия,Все,Япония кредитует Россию на полтора миллиарда д...,"['япония', 'принимать', 'решение', 'разморажив..."
3,8,1999-08-31,https://lenta.ru/news/1999/08/31/diana/,Мир,Все,Британцы отмечают двухлетие смерти Дианы,"['британец', 'отмечать', 'сегодня', 'скорбный'..."
4,9,1999-08-31,https://lenta.ru/news/1999/08/31/mvf/,Россия,Все,Отмытые через Bank of NY деньги не имели отнош...,"['понедельник', 'директор', 'департамент', 'вн..."
...,...,...,...,...,...,...,...
19990,19995,2001-01-17,https://lenta.ru/news/2001/01/17/case/,Россия,Все,Прокуратура Владивостока ищет виновных в отклю...,"['прокуратура', 'владивосток', 'возбуждать', '..."
19991,19996,2001-01-17,https://lenta.ru/news/2001/01/17/clinton/,Мир,Все,У Билла Клинтона диагностирован рак кожи,"['американский', 'президент', 'билл', 'клинтон..."
19992,19997,2001-01-17,https://lenta.ru/news/2001/01/17/truck/,Мир,Все,Грузовик протаранил и поджег дом правительства...,"['сакраменто', 'столица', 'штат', 'калифорния'..."
19993,19998,2001-01-17,https://lenta.ru/news/2001/01/17/duma/,Россия,Все,На открытии сессии Думы сидели только Сергей К...,"['среда', 'звук', 'новый', 'гимн', 'россия', '..."


Вроде все ок, отработало суммарно за 2 минуты, значит весь датафрейм обработается за ~2 часа

In [32]:
%%time

df_iterator = pd.read_csv("/Users/kucumovamilana/Downloads/lenta-ru-news-with-2022.csv", chunksize=10000)
new_chunk = 0
for df in df_iterator:
    new_chunk += 1
    print('Chunk =', new_chunk)
    # первые 2 выше уже прошлись
    if new_chunk <= 2:
        continue
    df['date'] = pd.to_datetime(df['date'])
    df = df[~df['text'].isna()]
    df = df[df['date'].dt.year != 1914]
    print(df.shape)
    df['text'] = df['text'].parallel_apply(make_lemmas)
    df.to_csv('lenta_news_with_lemmas.csv', mode='a', header=False)

Chunk = 1
Chunk = 2
Chunk = 3
Chunk = 4
Chunk = 5
Chunk = 6
Chunk = 7
Chunk = 8
Chunk = 9
Chunk = 10
Chunk = 11
Chunk = 12
Chunk = 13
Chunk = 14
Chunk = 15
Chunk = 16
(9996, 6)


Chunk = 17
(10000, 6)


Chunk = 18
(10000, 6)


Chunk = 19
(10000, 6)


Chunk = 20
(10000, 6)


...

Chunk = 113
(9993, 6)


Chunk = 114
(9987, 6)


Chunk = 115
(9993, 6)


Chunk = 116
(6511, 6)


CPU times: user 6min 19s, sys: 1min 14s, total: 7min 33s
Wall time: 2h 16min 19s


Проверим что получилось

In [2]:
df = pd.read_csv('lenta_news_with_lemmas.csv')

In [3]:
df

,Unnamed: 0,date,url,topic,tags,title,text
0,5,1999-08-31,https://lenta.ru/news/1999/08/31/stancia_mir/,Россия,Все,"Космонавты сомневаются в надежности ""Мира""","['становиться', 'известно', 'агентство', 'ассо..."
1,6,1999-08-31,https://lenta.ru/news/1999/08/31/vzriv/,Россия,Все,Взрыв в центре Москвы: пострадало 30 человек,"['зал', 'игровой', 'автомат', 'третий', 'ярус'..."
2,7,1999-08-31,https://lenta.ru/news/1999/08/31/credit_japs/,Россия,Все,Япония кредитует Россию на полтора миллиарда д...,"['япония', 'принимать', 'решение', 'разморажив..."
3,8,1999-08-31,https://lenta.ru/news/1999/08/31/diana/,Мир,Все,Британцы отмечают двухлетие смерти Дианы,"['британец', 'отмечать', 'сегодня', 'скорбный'..."
4,9,1999-08-31,https://lenta.ru/news/1999/08/31/mvf/,Россия,Все,Отмытые через Bank of NY деньги не имели отнош...,"['понедельник', 'директор', 'департамент', 'вн..."
...,...,...,...,...,...,...,...
1155604,1156513,2022-11-18,https://lenta.ru/news/2022/11/18/bezhh/,Мир,Общество,Беженца из Украины избили после спора о нацистах,"['беженец', 'украина', 'избивать', 'город', 'к..."
1155605,1156514,2022-11-18,https://lenta.ru/news/2022/11/18/nurmagomedov/,Спорт,Бокс и ММА,Брат Нурмагомедова назвал цель выступления в B...,"['российский', 'боец', 'смешанный', 'стиль', '..."
1155606,1156515,2022-11-18,https://lenta.ru/news/2022/11/18/bioprint/,Наука и техника,Будущее,Разработана технология печати ткани для трансп...,"['ученый', 'израильский', 'технологический', '..."
1155607,1156516,2022-11-18,https://lenta.ru/news/2022/11/18/penni/,Мир,Политика,Республиканцы в Конгрессе потребовали проверит...,"['республиканец', 'палата', 'представитель', '..."


При желании можем по старым индексам присоеднить оригинальные тексты

## Мешок слов

### Общий

Посмотрим на общий мешок слов и изучим самые часто встречающиеся слова

In [10]:
bag_of_words = Counter()

In [11]:
df_iterator = pd.read_csv("lenta_news_with_lemmas.csv", chunksize=50000)
chunk = 0
for df in df_iterator:
    chunk += 1
    print('Chunk =', chunk)
    print(df.shape)
    df['text'] = df['text'].parallel_apply(lambda x: x[2:-2].split("', '"))
    bag_of_words += Counter(list(itertools.chain(*list(df['text']))))

Chunk = 1
(50000, 7)


Chunk = 2
(50000, 7)


Chunk = 3
(50000, 7)


Chunk = 4
(50000, 7)


Chunk = 5
(50000, 7)


...

Chunk = 20
(50000, 7)


Chunk = 21
(50000, 7)


Chunk = 22
(50000, 7)


Chunk = 23
(50000, 7)


Chunk = 24
(5609, 7)


In [15]:
sorted(bag_of_words.items(), key=lambda item: item[1], reverse=True)[:20]

[('год', 1782742),
 ('который', 1379435),
 ('это', 1351046),
 ('россия', 886914),
 ('сообщать', 865085),
 ('свой', 677716),
 ('также', 676145),
 ('заявлять', 566810),
 ('человек', 553360),
 ('время', 535517),
 ('российский', 522626),
 ('становиться', 514772),
 ('страна', 514679),
 ('слово', 482852),
 ('процент', 473653),
 ('мочь', 464621),
 ('компания', 437754),
 ('весь', 431550),
 ('президент', 412912),
 ('ранее', 408554)]

### Мешок слов по топику

In [36]:
df_topic = pd.DataFrame(columns=['topic', 'bag_of_words'])

def bag_of_words_bytopic(topic):
    df_topic_f = df[df['topic'] == topic]

    list_of_news = list(df_topic_f['text'])
    union_list_of_news = list(itertools.chain(*list_of_news))
    
    bow_counter = Counter(union_list_of_news)
    
    return bow_counter

In [37]:
%%time

df_iterator = pd.read_csv("lenta_news_with_lemmas.csv", chunksize=50000)
chunk = 0
for df in df_iterator:
    chunk += 1
    print('Chunk =', chunk)
    print(df.shape)
    df['text'] = df['text'].parallel_apply(lambda x: x[2:-2].split("', '"))
    for topic in df['topic'].unique():
        bow_counter = bag_of_words_bytopic(topic)
        if topic not in list(df_topic['topic']):
            df_topic.at[len(df_topic), 'topic'] = topic
            df_topic.at[len(df_topic)-1, 'bag_of_words'] = bow_counter
        else:
            df_topic.loc[df_topic['topic'] == topic, 'bag_of_words'] += bow_counter

Chunk = 1
(50000, 7)


Chunk = 2
(50000, 7)


Chunk = 3
(50000, 7)


Chunk = 4
(50000, 7)


Chunk = 5
(50000, 7)


...

Chunk = 21
(50000, 7)


Chunk = 22
(50000, 7)


Chunk = 23
(50000, 7)


Chunk = 24
(5609, 7)


CPU times: user 3min 25s, sys: 2min 1s, total: 5min 26s
Wall time: 9min 28s


In [85]:
df_topic.dropna(inplace=True)

for topic in df_topic['topic'].unique():
    bag_of_words = df_topic.loc[df_topic['topic'] == topic, 'bag_of_words'].values[0]
    bag_of_words = sorted(bag_of_words.items(), key=lambda item: item[1], reverse=True)
    # сделаем просто строкой, чтобы быстро посмотреть
    df_topic.loc[df_topic['topic'] == topic, 'bag_of_words'] = str(bag_of_words)

In [127]:
df_topic.head()

,topic,bag_of_words
0,Россия,"[('год', 276206), ('это', 264579), ('который',..."
1,Мир,"[('который', 225060), ('год', 220484), ('это',..."
2,Экономика,"[('год', 319099), ('процент', 212676), ('компа..."
3,Интернет и СМИ,"[('который', 95073), ('год', 89031), ('это', 7..."
4,Спорт,"[('матч', 106388), ('год', 106051), ('это', 85..."


Посмотрим подробнее на несколько самых популярных слов

In [126]:
for i in range(df_topic.shape[0]):
    print('---------------------------------------------')
    print(df_topic['topic'].values[i])
    print(df_topic['bag_of_words'].values[i][:200])

---------------------------------------------
Россия
[('год', 276206), ('это', 264579), ('который', 255683), ('россия', 243292), ('сообщать', 201972), ('человек', 161371), ('также', 140489), ('заявлять', 125622), ('слово', 122765), ('москва', 117037), (
---------------------------------------------
Мир
[('который', 225060), ('год', 220484), ('это', 208858), ('сообщать', 173572), ('сша', 171577), ('страна', 158691), ('россия', 134058), ('заявлять', 133062), ('человек', 124477), ('свой', 120821), ('пр
---------------------------------------------
Экономика
[('год', 319099), ('процент', 212676), ('компания', 165371), ('это', 164354), ('россия', 158533), ('который', 139608), ('доллар', 134591), ('рубль', 113041), ('миллиард', 109822), ('российский', 10096
---------------------------------------------
Интернет и СМИ
[('который', 95073), ('год', 89031), ('это', 78078), ('свой', 52752), ('компания', 48071), ('также', 44326), ('пользователь', 43233), ('сообщать', 40890), ('сайт', 37360), ('рос

### Мешок слов по годам и месяцам

In [89]:
df_year = pd.DataFrame(columns=['year', 'bag_of_words'])
df_month = pd.DataFrame(columns=['month', 'bag_of_words'])

def bag_of_words_by_year(year, df):
    df_f = df[df['date'].dt.year == year]

    list_of_news = list(df_f['text'])
    union_list_of_news = list(itertools.chain(*list_of_news))
    
    bow_counter = Counter(union_list_of_news)
    return bow_counter

def bag_of_words_by_month(month, df):
    df_f = df[df['date'].dt.month == month]

    list_of_news = list(df_f['text'])
    union_list_of_news = list(itertools.chain(*list_of_news))
    
    bow_counter = Counter(union_list_of_news)
    return bow_counter

In [90]:
%%time

df_iterator = pd.read_csv("lenta_news_with_lemmas.csv", chunksize=50000)
chunk = 0
for df in df_iterator:
    chunk += 1
    print('Chunk =', chunk)
    print(df.shape)
    df['date'] = pd.to_datetime(df['date'])
    df['text'] = df['text'].parallel_apply(lambda x: x[2:-2].split("', '"))
    for year in df['date'].dt.year.unique():
        bow_counter = bag_of_words_by_year(year, df)
        if year not in list(df_year['year']):
            df_year.at[len(df_year), 'year'] = year
            df_year.at[len(df_year)-1, 'bag_of_words'] = bow_counter
        else:
            df_year.loc[df_year['year'] == year, 'bag_of_words'] += bow_counter
            
    for month in df['date'].dt.month.unique():
        bow_counter = bag_of_words_by_month(month, df)
        if month not in list(df_month['month']):
            df_month.at[len(df_month), 'month'] = month
            df_month.at[len(df_month)-1, 'bag_of_words'] = bow_counter
        else:
            df_month.loc[df_month['month'] == month, 'bag_of_words'] += bow_counter

Chunk = 1
(50000, 7)


Chunk = 2
(50000, 7)


Chunk = 3
(50000, 7)


Chunk = 4
(50000, 7)


Chunk = 5
(50000, 7)


...

Chunk = 21
(50000, 7)


Chunk = 22
(50000, 7)


Chunk = 23
(50000, 7)


Chunk = 24
(5609, 7)


CPU times: user 4min 30s, sys: 2min 49s, total: 7min 19s
Wall time: 12min 42s


In [100]:
for year in df_year['year'].unique():
    bag_of_words = df_year.loc[df_year['year'] == year, 'bag_of_words'].values[0]
    bag_of_words = sorted(bag_of_words.items(), key=lambda item: item[1], reverse=True)
    df_year.loc[df_year['year'] == year, 'bag_of_words'] = str(bag_of_words)
df_year

,year,bag_of_words
0,1999,"[('сообщать', 3100), ('год', 2919), ('который'..."
1,2000,"[('который', 12933), ('сообщать', 12870), ('го..."
2,2001,"[('сообщать', 20104), ('который', 18698), ('го..."
3,2002,"[('сообщать', 22839), ('который', 22774), ('го..."
4,2003,"[('который', 23424), ('сообщать', 21395), ('го..."
5,2004,"[('который', 29939), ('год', 29077), ('сообщат..."
6,2005,"[('год', 43331), ('который', 39228), ('сообщат..."
7,2006,"[('год', 48782), ('который', 40688), ('сообщат..."
8,2007,"[('год', 53988), ('который', 39964), ('сообщат..."
9,2008,"[('год', 80839), ('который', 56126), ('сообщат..."


In [128]:
for i in range(df_year.shape[0]):
    print('---------------------------------------------')
    print(df_year['year'].values[i])
    print(df_year['bag_of_words'].values[i][:200])

---------------------------------------------
1999
[('сообщать', 3100), ('год', 2919), ('который', 2752), ('россия', 2592), ('это', 2035), ('российский', 1755), ('время', 1587), ('весь', 1542), ('свой', 1527), ('также', 1474), ('заявлять', 1452), ('че
---------------------------------------------
2000
[('который', 12933), ('сообщать', 12870), ('год', 11545), ('это', 8858), ('россия', 8334), ('свой', 7063), ('также', 5929), ('время', 5873), ('российский', 5839), ('заявлять', 5753), ('человек', 5648)
---------------------------------------------
2001
[('сообщать', 20104), ('который', 18698), ('год', 17880), ('это', 12360), ('свой', 11404), ('россия', 9537), ('заявлять', 8475), ('время', 8403), ('человек', 7984), ('весь', 7726), ('также', 7655), ('
---------------------------------------------
2002
[('сообщать', 22839), ('который', 22774), ('год', 21007), ('россия', 12733), ('это', 12258), ('заявлять', 11950), ('свой', 11891), ('время', 9884), ('человек', 9732), ('также', 8810), ('весь', 

In [101]:
for month in df_month['month'].unique():
    bag_of_words = df_month.loc[df_month['month'] == month, 'bag_of_words'].values[0]
    bag_of_words = sorted(bag_of_words.items(), key=lambda item: item[1], reverse=True)
    df_month.loc[df_month['month'] == month, 'bag_of_words'] = str(bag_of_words)
df_month

,month,bag_of_words
0,8,"[('год', 143896), ('который', 121081), ('это',..."
1,9,"[('год', 148421), ('это', 120993), ('который',..."
2,10,"[('год', 159265), ('который', 128348), ('это',..."
3,11,"[('год', 151287), ('который', 118302), ('это',..."
4,12,"[('год', 164497), ('который', 114488), ('это',..."
5,1,"[('год', 145883), ('который', 98847), ('это', ..."
6,2,"[('год', 145676), ('который', 104410), ('это',..."
7,3,"[('год', 149328), ('который', 114868), ('это',..."
8,4,"[('год', 148332), ('который', 114976), ('это',..."
9,5,"[('год', 134890), ('который', 108347), ('это',..."


In [131]:
df_month.sort_values(by=['month'], inplace=True)

In [132]:
for i in range(df_month.shape[0]):
    print('---------------------------------------------')
    print(df_month['month'].values[i])
    print(df_month['bag_of_words'].values[i][:200])

---------------------------------------------
1
[('год', 145883), ('который', 98847), ('это', 95402), ('январь', 65091), ('сообщать', 62351), ('россия', 57395), ('свой', 50169), ('также', 47725), ('человек', 40161), ('заявлять', 40004), ('время', 3
---------------------------------------------
2
[('год', 145676), ('который', 104410), ('это', 102624), ('россия', 69527), ('сообщать', 66645), ('февраль', 65056), ('свой', 53489), ('также', 51154), ('заявлять', 45293), ('время', 41197), ('человек'
---------------------------------------------
3
[('год', 149328), ('который', 114868), ('это', 112068), ('россия', 81306), ('сообщать', 73421), ('март', 68766), ('также', 57697), ('свой', 56851), ('российский', 48453), ('заявлять', 48382), ('челове
---------------------------------------------
4
[('год', 148332), ('который', 114976), ('это', 112588), ('россия', 79118), ('сообщать', 73439), ('апрель', 72242), ('также', 56481), ('свой', 56427), ('человек', 48756), ('заявлять', 48376), ('российс
----

## Выводы

Для анализа наиболее популярных слов в разных разрезах необходимо было сделать так называемый «мешок слов» - количество употребления в тексте каждого слова. Мы сделали мешки слов по всему датафрейму, по топикам, по годам и месяцам.

Результаты получились следующие:

- На всем датафрейме наиболее популярные слова, на наш взгляд, не несут довольно большого смысла (слова «год», «это», «который»)


- Гораздо интереснее посмотреть на популярные слова в каждом топике. Так, например, в топике «Спорт» популярны слова «матч», «команда», «сборная». На наш взгляд у таких топиков будут хорошие метрики  при применении моделей классификации. В то же время у некоторых популярные слова довольно похожи, например у топиков «Экономика» и «Бизнес» популярны слова «процент», «компания», «миллиард», «рубль». Здесь мы ожидаем получить более худшее качество по метрикам, хотя возможно при сочетании с другими словами, оно будет лучше.


- Мешок слов по годам показался также довольно информативным. Так, например, одно из популярных слов в 2020 году было слово «коронавирус», в 2014 и 2022 - «Украина». 


- По месяцам оказался наименее информативный мешок слов, так как топ-10 слов практически идентичен у каждого месяца.